In [2]:
import pandas as pd
import glob

In [33]:
df = pd.read_csv('validation-set-sentence-labelled.tsv', sep='\t')

In [69]:
df['doc-sent'] = [str(row['document_ID']) + '-' + str(row['sentence_ID']) for index, row in df.iterrows()]
df['structure_focus'].fillna(False, inplace=True)
df['process_focus'].fillna(False, inplace=True)
df['usecase_focus'].fillna(False, inplace=True)

In [59]:
class Bucketizer:    
    def is_important_for_class(self, supersense_df):
        # Rule 1: there needs to be at least two nouns in the sentences
        if supersense_df['fine_POS_tag'].tolist().count('NN') + supersense_df['fine_POS_tag'].tolist().count('NNP') + supersense_df['fine_POS_tag'].tolist().count('NNS') > 1:
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['aux', 'auxpass', 'conj']) and 'verb.cognition' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['pobj', 'prep', 'nsubj']) and all(item in supersense_df['supersense_category'].tolist() for item in ['verb.stative', 'noun.relation']):
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['pobj', 'nummod']) and all(item in supersense_df['supersense_category'].tolist() for item in ['verb.stative', 'noun.artifact']) and supersense_df['supersense_category'].tolist().count('verb.stative') > 1:
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['nsubj', 'dobj', 'amod']) and 'verb.stative' in supersense_df['supersense_category'].tolist() and 'JJ' in supersense_df['fine_POS_tag'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'dobj']) and 'verb.possession' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'pobj', 'aux']) and 'verb.stative' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'pobj', 'aux']) and 'verb.communication' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'dobj', 'cc']) and 'verb.change' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'dobj']) and all(item in supersense_df['supersense_category'].tolist() for item in ['verb.perception', 'noun.artifact']):
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['prep', 'nsubjpass', 'auxpass', 'nummod', 'cc']):
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['nsubjpass', 'auxpass']) and 'require' in supersense_df['lemma'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['nsubjpass', 'auxpass', 'det', 'aux']) and 'verb.change' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubjpass', 'aux', 'auxpass', 'pobj']) and 'verb.contact' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'pobj', 'prep']) and 'verb.stative' in supersense_df['supersense_category'].tolist():
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['pobj', 'prep']) and all(item in supersense_df['supersense_category'].tolist() for item in ['verb.stative', 'noun.artifact']) and supersense_df['dependency_relation'].tolist().count('pobj') > 1:
                return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['det', 'nsubj', 'dobj', 'predet']) and 'verb.social' in supersense_df['supersense_category'].tolist():
                return True
            else:
                return False
        else:
            return False
    
    def is_important_for_activity(self, supersense_df):
        # Rule 1: there needs to be at least two nouns in the sentences
        if supersense_df['fine_POS_tag'].tolist().count('NN') + supersense_df['fine_POS_tag'].tolist().count('NNP') + supersense_df['fine_POS_tag'].tolist().count('NNS') > 1:
            # if all(item in supersense_df['dependency_relation'].tolist() for item in ['nsubj', 'aux', 'cc', 'conj']) and any(item in supersense_df['supersense_category'].tolist() for item in ['verb.cognition', 'verb.creation']):
            #     print('!?')
            #     return True
            if all(item in supersense_df['dependency_relation'].tolist() for item in ['nsubj', 'acl', 'agent', 'ccomp']) and all(item in supersense_df['supersense_category'].tolist() for item in ['verb.communication', 'verb.stative']):
                return True
            if len(supersense_df[(supersense_df['lemma'].isin(['when', 'second', 'if', 'then', 'first'])) & (supersense_df['dependency_relation'] == 'advmod')]) > 0:
                return True
            else: 
                return False
        else:
            return False
    
    def is_important_for_usecase(self, supersense_df):
        # Rule 1: there needs to be at least two nouns in the sentences
        if supersense_df['fine_POS_tag'].tolist().count('NN') + supersense_df['fine_POS_tag'].tolist().count('NNP') + supersense_df['fine_POS_tag'].tolist().count('NNS') > 1:
            if len(supersense_df[supersense_df['lemma'].isin(['system', 'facilitate', 'module', 'interface', 'functionality', 'capability'])]) > 0:
                return True
            if len(supersense_df[supersense_df['supersense_category'] == 'verb.contact']) > 0:
                return True
            if len(supersense_df[(supersense_df['supersense_category'].isin(['verb.creation', 'verb.social'])) & (supersense_df['lemma'] == 'execute')]) > 0:
                return True
            if len(supersense_df[(supersense_df['supersense_category'].isin(['noun.animal', 'noun.person', 'noun.plant'])) & (supersense_df['dependency_relation'] == 'nsubj')]) > 0:
                return True
            else:
                return False
        else:
            return False

    def apply_bucketing(self, input_text):
        return [
            self.is_important_for_class(input_text),
            self.is_important_for_activity(input_text),
            self.is_important_for_usecase(input_text)
        ]

bucketizer = Bucketizer()        

In [63]:
pred = df.groupby('doc-sent').apply(lambda x: bucketizer.apply_bucketing(x))

In [70]:
test = df.groupby('doc-sent').apply(lambda x: [x.iloc[0]['structure_focus'], x.iloc[0]['process_focus'], x.iloc[0]['usecase_focus']])

In [76]:
from sklearn.metrics import classification_report

In [71]:
test

doc-sent
0-0       [True, False, True]
0-1        [True, True, True]
0-10       [True, True, True]
0-11    [False, False, False]
0-12       [True, True, True]
                ...          
4-5      [True, False, False]
4-6        [True, True, True]
4-7      [True, False, False]
4-8     [False, False, False]
4-9      [True, False, False]
Length: 145, dtype: object

In [72]:
pred

doc-sent
0-0      [True, False, False]
0-1       [True, False, True]
0-10    [False, False, False]
0-11    [False, False, False]
0-12     [True, False, False]
                ...          
4-5     [False, False, False]
4-6       [True, True, False]
4-7      [True, False, False]
4-8     [False, False, False]
4-9       [True, False, True]
Length: 145, dtype: object

In [78]:
print('Classes:')
print(classification_report([observation[0] for observation in test], [observation[0] for observation in pred]))

print('Activity:')
print(classification_report([observation[1] for observation in test], [observation[1] for observation in pred]))

print('Use case:')
print(classification_report([observation[2] for observation in test], [observation[2] for observation in pred]))

Classes:
              precision    recall  f1-score   support

       False       0.28      0.54      0.37        35
        True       0.79      0.56      0.66       110

    accuracy                           0.56       145
   macro avg       0.54      0.55      0.52       145
weighted avg       0.67      0.56      0.59       145

Activity:
              precision    recall  f1-score   support

       False       0.90      0.95      0.92       123
        True       0.60      0.41      0.49        22

    accuracy                           0.87       145
   macro avg       0.75      0.68      0.71       145
weighted avg       0.85      0.87      0.86       145

Use case:
              precision    recall  f1-score   support

       False       0.87      0.83      0.85       118
        True       0.38      0.44      0.41        27

    accuracy                           0.76       145
   macro avg       0.62      0.64      0.63       145
weighted avg       0.78      0.76      0.77  

In [79]:
len(pred)

145